In [1]:
import torch
from model import load_resnet_model
from train_regression_weighted_loss_loaded import train_model
from dataloader import create_dataloader

from utils import sorted_file_paths
import torch.nn as nn
import torch.optim as optim

# Setting up common variables for all experiments

In [2]:
LR = 0.001
EPOCH = 1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_files = sorted_file_paths('./data/records/train/')
val_files = sorted_file_paths('./data/records/val/')

train_loaders = [create_dataloader(file, True, 64) for file in train_files]
val_loaders = [create_dataloader(file, True, 64) for file in val_files]

training_labels = []
for train_loader in train_loaders:
    for _, labels in train_loader:
        training_labels.extend(labels.detach().cpu().numpy())

# Experiment 1: L2 loss

In [3]:
ex = 'ex_1'
loss =  nn.MSELoss()
model = load_resnet_model('resnet50', num_classes=1)
optimizer = optim.Adam(model.parameters(), LR)

train_model(model, loss, optimizer, train_loaders, val_loaders, device, ex, epochs=EPOCH)

c:\Users\lhmtr\OneDrive\Desktop\remote-sensing-asset-index\myenv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\lhmtr\OneDrive\Desktop\remote-sensing-asset-index\myenv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/1, Train Loss: 1.8711, Train R2: -8.3076, Train weighted MAE: 1.0176,Val Loss: 92.0891, Val R2: -500.8802, Val weighted MAE: 6.0679
Model saved as ex_1/checkpoint_epoch_1.pth
Model saved as ex_1/final_model.pth
Training completed and final model saved.


# Experiment 2: L3 loss

In [4]:
from weighted_L3_loss import L3Loss

ex = 'ex_2'
loss =  L3Loss()
model = load_resnet_model('resnet50', num_classes=1)
optimizer = optim.Adam(model.parameters(), LR)

train_model(model, loss, optimizer, train_loaders, val_loaders, device, ex, epochs=EPOCH)

c:\Users\lhmtr\OneDrive\Desktop\remote-sensing-asset-index\myenv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\lhmtr\OneDrive\Desktop\remote-sensing-asset-index\myenv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/1, Train Loss: 5.5348, Train R2: -9.1135, Train weighted MAE: 0.9844,Val Loss: 7149232.9531, Val R2: -66035.3811, Val weighted MAE: 23.7026


RuntimeError: Parent directory ex_2 does not exist.

# Experiment 3: Weighted MSE with dense_weight model, alpha = 0.25

In [8]:
from dense_weight import DenseWeight
from weighted_MSE_loss import WeightedMSELoss

In [9]:
ex = 'ex_3'

dense_weight_model = DenseWeight(0.25)
dense_weight_model.fit(training_labels)
loss =  WeightedMSELoss(dense_weight_model)
model = load_resnet_model('resnet50', num_classes=1)
optimizer = optim.Adam(model.parameters(), LR)

train_model(model, loss, optimizer, train_loaders, val_loaders, device, ex, epochs=EPOCH)

c:\Users\lhmtr\OneDrive\Desktop\remote-sensing-asset-index\myenv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\lhmtr\OneDrive\Desktop\remote-sensing-asset-index\myenv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MAE for bin 2: 1.5641959
MAE for bin 3: 1.1126256
MAE for bin 4: 1.0491617
MAE for bin 5: 0.9847947
MAE for bin 6: 0.9906604
MAE for bin 1: 60.09752
MAE for bin 2: 50.60432
MAE for bin 3: 54.209522
MAE for bin 4: 58.770176
MAE for bin 5: 61.25448
MAE for bin 6: 61.864296
Epoch 1/1, Train Loss: 2.0621, Train R2: -9.1192, Train weighted MAE: 1.4322,Val Loss: 4109.0274, Val R2: -22487.6166, Val weighted MAE: 58.1779
Model saved as checkpoint_epoch_1.pth
Model saved as final_model.pth
Training completed and final model saved.


# Experiment 4: Weighted MSE with dense_weight model, alpha = 0.5

In [10]:
ex = 'ex_4'

dense_weight_model = DenseWeight(0.5)
dense_weight_model.fit(training_labels)
loss =  WeightedMSELoss(dense_weight_model)
model = load_resnet_model('resnet50', num_classes=1)
optimizer = optim.Adam(model.parameters(), LR)

train_model(model, loss, optimizer, train_loaders, val_loaders, device, ex, epochs=EPOCH)

c:\Users\lhmtr\OneDrive\Desktop\remote-sensing-asset-index\myenv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\lhmtr\OneDrive\Desktop\remote-sensing-asset-index\myenv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MAE for bin 2: 1.1206676
MAE for bin 3: 0.9540928
MAE for bin 4: 0.9106966
MAE for bin 5: 0.96642876
MAE for bin 6: 1.1671207
MAE for bin 1: 10.19127
MAE for bin 2: 19.229319
MAE for bin 3: 13.944123
MAE for bin 4: 13.980035
MAE for bin 5: 17.688894
MAE for bin 6: 25.890987
Epoch 1/1, Train Loss: 1.6738, Train R2: -7.6164, Train weighted MAE: 1.0850,Val Loss: 900.6723, Val R2: -5656.7559, Val weighted MAE: 12.1556
Model saved as checkpoint_epoch_1.pth
Model saved as final_model.pth
Training completed and final model saved.


# Experiment 5: Balanced MSE loss

In [11]:
from balanced_MSE_loss import BMCLoss

In [12]:
ex = 'ex_5'

init_noise_sigma = 1.0
sigma_lr = 0.001
loss = BMCLoss(init_noise_sigma)
model = load_resnet_model('resnet50', num_classes=1)
optimizer = optim.Adam(model.parameters(), lr=LR)
optimizer.add_param_group({'params': loss.noise_sigma, 'lr': sigma_lr, 'name': 'noise_sigma'})

train_model(model, loss, optimizer, train_loaders, val_loaders, device, ex, epochs=EPOCH)

c:\Users\lhmtr\OneDrive\Desktop\remote-sensing-asset-index\myenv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\lhmtr\OneDrive\Desktop\remote-sensing-asset-index\myenv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MAE for bin 2: 1.4273541
MAE for bin 3: 1.1952238
MAE for bin 4: 1.7503417
MAE for bin 5: 1.7749552
MAE for bin 6: 2.318867
MAE for bin 1: 2.951308
MAE for bin 2: 1.9054776
MAE for bin 3: 4.6486855
MAE for bin 4: 4.2353673
MAE for bin 5: 2.5514913
MAE for bin 6: 2.5924032
Epoch 1/1, Train Loss: 8.6341, Train R2: -29.0568, Train weighted MAE: 1.4724,Val Loss: 11.1934, Val R2: -140.1235, Val weighted MAE: 2.7783
Model saved as checkpoint_epoch_1.pth
Model saved as final_model.pth
Training completed and final model saved.
